Proof of <a class="ProveItLink" href="../../../../../../_theory_nbs_/theory.ipynb">proveit</a>.<a class="ProveItLink" href="../../../../../_theory_nbs_/theory.ipynb">physics</a>.<a class="ProveItLink" href="../../../../_theory_nbs_/theory.ipynb">quantum</a>.<a class="ProveItLink" href="../../theory.ipynb">QPE</a>.<a class="ProveItLink" href="../../theorems.ipynb#_fail_sum">_fail_sum</a> theorem
========

In [ ]:
import proveit
theory = proveit.Theory() # the theorem's theory

from proveit import defaults, Lambda
from proveit import a, b, c, e, f, l, m, x, y, A, B, C, N, Q, X, Omega
from proveit.logic import Or, Forall, InSet, Union, Disjoint
from proveit.logic.sets.functions.injections import subset_injection
from proveit.logic.sets.functions.bijections import bijection_transitivity
from proveit.numbers import (zero, one, Add, Neg, greater, Less)
from proveit.numbers.modular import interval_left_shift_bijection
from proveit.statistics import prob_of_disjoint_events_is_prob_sum
from proveit.physics.quantum import NumKet
from proveit.physics.quantum.QPE import (
    _t, _best_floor_is_int, _e_domain_sorting,
    _two_pow_t, _two_pow_t__minus_one, _b_floor, _t_in_natural_pos,
    _two_pow_t_minus_one_is_nat_pos, _best_floor_def, _Omega,
    _alpha_m_def, _full_domain, _neg_domain, _pos_domain,
    _modabs_in_full_domain_simp, ModAdd, _sample_space_bijection,
    _Omega_is_sample_space, _outcome_prob, _fail_def,
    _fail_sum_prob_conds_equiv_lemma)

In [ ]:
%proving _fail_sum

In [ ]:
defaults.assumptions = _fail_sum.all_conditions()

In [ ]:
e_in_domain = defaults.assumptions[0]

In [ ]:
defaults.preserved_exprs = {e_in_domain.domain.upper_bound.operands[0]}

## Start with the $P_{\textrm{fail}}$ definition

In [ ]:
fail_def_inst = _fail_def.instantiate()

## We will next translate from $m \in \{0~\ldotp \ldotp~2^{t} - 1\}$ to $b_f \oplus l$ with $l \in \{-2^{t-1} + 1 ~\ldotp \ldotp~2^{t-1}\}$

In [ ]:
lower_bound = _neg_domain.lower_bound

In [ ]:
upper_bound = _pos_domain.upper_bound

In [ ]:
interval_left_shift_bijection

In [ ]:
lambda_map_bijection = interval_left_shift_bijection.instantiate(
    {a:lower_bound, b:upper_bound, c:_b_floor, N:_two_pow_t, x:l})

Use the $\oplus$ notation within kets (but not otherwise, for simplification purposed):

In [ ]:
b_modadd_l__def = ModAdd(_b_floor, l).definition(assumptions=[InSet(l, _full_domain)])

In [ ]:
defaults.replacements = [b_modadd_l__def.derive_reversed().substitution(NumKet(b_modadd_l__def.rhs, _t))]

In [ ]:
transformed_fail_def = fail_def_inst.inner_expr().rhs.transform(
    lambda_map_bijection.element, lambda_map_bijection.domain.domain, _Omega)

## Proceed to split the probability of this event into the sum of two disjoint possibilities

Our goal will be to apply the following theorem:

In [ ]:
prob_of_disjoint_events_is_prob_sum

### Prove the $|l|_{\textrm{mod}~2^t} > e$ condition for either the positive or negative domain for simplification purposes

In [ ]:
condition = transformed_fail_def.rhs.non_domain_condition()

In [ ]:
neg_domain_assumptions = [InSet(l, _neg_domain), *_fail_sum.all_conditions()]

In [ ]:
pos_domain_assumptions = [InSet(l, _pos_domain), *_fail_sum.all_conditions()]

In [ ]:
_e_domain_sorting

In [ ]:
_e_domain_sorting.instantiate()

In [ ]:
_modabs_in_full_domain_simp

In [ ]:
absmod_posdomain_simp = _modabs_in_full_domain_simp.instantiate(
    assumptions=pos_domain_assumptions)

In [ ]:
absmod_negdomain_simp = _modabs_in_full_domain_simp.instantiate(
    assumptions=neg_domain_assumptions)

In [ ]:
pos_cond_eval = absmod_posdomain_simp.sub_left_side_into(
    greater(l, e), assumptions=pos_domain_assumptions).evaluation()

In [ ]:
neg_cond_eval = absmod_negdomain_simp.sub_left_side_into(
    greater(Neg(l), e), assumptions=neg_domain_assumptions).evaluation()

### Prove the proper injective mapping condition then apply `prob_of_disjoint_events_is_prob_sum`

In [ ]:
circuit_map = Lambda(l, transformed_fail_def.rhs.instance_expr)

In [ ]:
lambda_map_bijection

In [ ]:
_sample_space_bijection

In [ ]:
trans_bijection = lambda_map_bijection.apply_transitivity(_sample_space_bijection)

In [ ]:
prob_eq_sum1 = prob_of_disjoint_events_is_prob_sum.instantiate(
    {Omega:_Omega, A:_neg_domain, B:_pos_domain, C:Union(_neg_domain, _pos_domain), 
     X:_full_domain, f:circuit_map, Q:Lambda(l, condition), x:l},
    replacements=[pos_cond_eval, neg_cond_eval]).with_wrap_after_operator()

## Compute the event probabilities in terms of summing outcome probabilities

In [ ]:
outcome_prob_pos = _outcome_prob.instantiate({m:ModAdd(_b_floor, l)}, assumptions=pos_domain_assumptions)

In [ ]:
outcome_prob_neg = _outcome_prob.instantiate({m:ModAdd(_b_floor, l)}, assumptions=neg_domain_assumptions)

In [ ]:
subset_injection

In [ ]:
subset_injection.instantiate({A:_pos_domain, B:_full_domain, C:_Omega, f:trans_bijection.element})

In [ ]:
subset_injection.instantiate({A:_neg_domain, B:_full_domain, C:_Omega, f:trans_bijection.element})

In [ ]:
prob_eq_sum2 = prob_eq_sum1.inner_expr().rhs.operands[0].compute(_Omega, replacements=[outcome_prob_neg])

In [ ]:
prob_eq_sum3 = prob_eq_sum2.inner_expr().rhs.operands[1].compute(_Omega, replacements=[outcome_prob_pos])

That logical equivalence of conditions is handled in a separate lemma, ``_fail_sum_prob_conds_equiv_lemma``:

In [ ]:
_fail_sum_prob_conds_equiv_lemma

In [ ]:
# Here we explicitly preserve the -(e+1) expression; otherwise it becomes (-e-1) and
# later won't match up with the expression to be substituted for
conditions_equiv = (
    _fail_sum_prob_conds_equiv_lemma.instantiate(preserve_expr=Neg(Add(e, one)))
    .instantiate(preserve_expr=Neg(Add(e, one)), sideeffect_automation=False))

In [ ]:
prob_eq_sum4 = prob_eq_sum3.inner_expr().lhs.operand.body

In [ ]:
prob_eq_sum4 = prob_eq_sum3.inner_expr().lhs.operand.body.substitute_condition(
    conditions_equiv, preserve_all=True, sideeffect_automation=False)

In [ ]:
%qed